## Drug-target interactions: Predicting pEC50 for molecules binding to dopamine receptors

#### This is a full-blown machine learning project to predict the drug-target binding characteristics of small molecules that are interacting with different dopamine receptors. 

#### Considering all dopamine receptor types (D1, D2, D3, D4 and D5), our desired dataset is first generated which contains agonist molecules and their EC50 values (or pEC50) with respect to these receptors. Then, the project proceeds to perform machine learning analysis for pEC50 prediction.

#### Course of action: 

1. Create a dataset of small molecules, their respective dopamine target recptors and the EC50 values using ChEMBL. 
    * Cleaning our data.
    * Make sure our target label is pEC50.
2. The dataset will be enriched further via RDKit and the final data will be used for the project.
3. Perform Exploratory Data Analysis (EDA) and Machine Learning using this final dataset (performed in `dopamine_pEC50_ML.ipynb`).

### Let's begin!!

## 1: Generating our dataset.

### Below listed steps are followed for our dataset generation process.

* From the 5 different types of dopamine receptors in humans (D1, D2, D3, D4 and D5), I will collect the samples of small molecules that interact with these receptors and their respective EC50 (or pEC50) values, by querying from ChEMBL. The following additional information about the experimental conditions will also be extracted, if available,:
    * ChEMBL ID of the molecule
    * SMILES string
    * target information (will be known to me)
    * EC50 values (nM units)
    * pEC50 (if available)
    * pH
    * Temperature
    * Substrate concentrations 
* Using RDKit, I will then enrich the dataset using the SMILES information of the small molecules with other important features (that also count as Lipinski's parameters) such as
  * Molecular weight (MW)
  * Partition coefficient (LogP)
  * Number of H donors and acceptors
  * Topological Polar Surface Area (TPSA)
  * Ring count: Number of rings in the molecule.
  * Rotatable bonds: Number of bonds that can freely rotate.

### Extra information: Lipinski's rule of 5

The final enriched dataset is enriched with features that also help in drug classification problems. These are often called Lipinski descriptors - H bond donors, H bond acceptors, MW and logP.

The rule of 5 indicates that poor absorption of a drug/molecule is more likely to occur when there are more than 
1. 5 hydrogen-bond donors, 
2. 10 (5 × 2) hydrogen-bond acceptors, 
3. a molecular weight greater than 500 (5 × 100), and 
4. a calculated Log P (cLogP) greater than 5.

It is a rule of thumb that evaluates drug-likeness and determines whether a chemical compound with specific pharmacological activities has physical and chemical properties that would make it an orally active drug in humans. [ref](https://www.sciencedirect.com/topics/pharmacology-toxicology-and-pharmaceutical-science/lipinskis-rule-of-five#:~:text=The%20rule%20of%205%20indicates,(cLogP)%20greater%20than%205.)


In [ ]:
# !pip install jupyter_contrib_nbextensions

In [ ]:
# conda install -c conda-forge jupyter_contrib_nbextensions

In [ ]:
# !jupyter contrib nbextension install --user

In [ ]:
# necessary imports 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd 
import requests 

%matplotlib inline 

In [ ]:
# !pip install chembl_webresource_client

In [ ]:
from chembl_webresource_client.new_client import new_client

### Dopamine receptors and their ChEMBL IDs  

The ChEMBL ID information for receptors in humans is extracted from [this page](https://www.guidetopharmacology.org/GRAC/LigandActivityRangeVisForward?ligandId=940).

To begin, I will generate separate CSV files containg the information extracted for each receptor type and then combine them later on. 

The machine learning model will train for predicting the potency for the small molecules if the receptor is known.

| S. No. | Name | ChEMBL ID | Number of entries | 
| --- | --- | --- | --- | 
| 1. | D1 | CHEMBL2056 | 538 | 
| 2. | D2 | CHEMBL217 | 1892 | 
| 3. | D3 | CHEMBL234 | 764 |
| 4. | D4 | CHEMBL219 | 408 |
| 5. | D5 | CHEMBL1850 | 35 |

Note: I checked that the EC50 value is more readily available for these receptors rather than their pEC50. I will create another feature for pEC50 later on.

In [ ]:
# Initialize ChEMBL client
activity = new_client.activity

# List of dopamine receptor ChEMBL IDs
dopamine_receptor_ids = ["CHEMBL2056", "CHEMBL217", "CHEMBL234", "CHEMBL219", "CHEMBL1850"]

# Loop through each dopamine receptor and extract data
for target_id in dopamine_receptor_ids:
    # Filter for the desired target, and EC50
    activities = activity.filter(
        target_chembl_id=target_id,
        standard_type="EC50"  # Extract pEC50 values
    )

    print(target_id, len(activities))

CHEMBL2056 538
CHEMBL217 1892
CHEMBL234 764
CHEMBL219 408
CHEMBL1850 35


In [ ]:
# Initialize ChEMBL client
activity = new_client.activity

# List of dopamine receptor ChEMBL IDs
dopamine_receptor_ids = ["CHEMBL2056", "CHEMBL217", "CHEMBL234", "CHEMBL219", "CHEMBL1850"]

# Initialize lists to store data
chembl_ids = []
smiles_ids = []
EC50_values = []
EC50_units = []
assay_types = []
pH_values = []
temperatures = []
substrate_concs = []
target_name = []

# Loop through each dopamine receptor and extract data
for target_id in dopamine_receptor_ids:
    print(target_id)
    
    # Filter for the desired target, and EC50
    activities = activity.filter(
        target_chembl_id=target_id,
        standard_type="EC50"  # Extract pEC50 values
    )
    
    # Extract fields for each activity
    for i in range(len(activities)):
        # if (i%100==0): 
        #    print(target_id,i)
            
        act = activities[i]
        chembl_ids.append(act['molecule_chembl_id'])
        smiles_ids.append(act.get('canonical_smiles', None))
        EC50_values.append(act['standard_value'])  # EC50
        EC50_units.append(act.get('standard_units',None))
        assay_types.append(act.get('assay_type', None))
        pH_values.append(act.get('pH', None))  # pH value
        temperatures.append(act.get('temperature', None))  # Temperature
        substrate_concs.append(act.get('substrate_concentration', None))  # Substrate concentration
        target_name.append(act.get('target_pref_name',None))

# Create a DataFrame
data = {
    'ChEMBL ID': chembl_ids,
    'SMILES': smiles_ids,
    'EC50': EC50_values,
    'EC50 units': EC50_units,
    'Assay Type': assay_types,
    'pH': pH_values,
    'Temperature': temperatures,
    'Substrate Concentration': substrate_concs,
    'Target Name': target_name
}

df = pd.DataFrame(data)

# Display the DataFrame
df.head()


CHEMBL2056
CHEMBL217
CHEMBL234
CHEMBL219
CHEMBL1850


,ChEMBL ID,SMILES,EC50,EC50 units,Assay Type,pH,Temperature,Substrate Concentration,Target Name
0,CHEMBL87897,COc1ccc(CC2Cc3c(ccc(O)c3O)[C@H](CN)O2)cc1,237.0,nM,F,None,None,None,Dopamine D1 receptor
1,CHEMBL2115213,CC[C@@H]1Cc2c(ccc(O)c2O)[C@H](CN)O1,142.0,nM,F,None,None,None,Dopamine D1 receptor
2,CHEMBL420972,CCCCCCC1Cc2c(ccc(O)c2O)[C@H](CN)O1,13.6,nM,F,None,None,None,Dopamine D1 receptor
3,CHEMBL420788,C#CCNC[C@@H]1OC(C2CCCCC2)Cc2c1ccc(O)c2O,12.5,nM,F,None,None,None,Dopamine D1 receptor
4,CHEMBL87910,NC[C@@H]1OC(Cc2ccccc2)Cc2c1ccc(O)c2O,34.4,nM,F,None,None,None,Dopamine D1 receptor


In [ ]:
len(df)

3637

In [ ]:
# any missing data? 
df.isna().sum()

ChEMBL ID                     0
SMILES                       15
EC50                        375
EC50 units                  377
Assay Type                    0
pH                         3637
Temperature                3637
Substrate Concentration    3637
Target Name                   0
dtype: int64

In [ ]:
# since we don't have pH, Temperature and substrate concentrations available, we must remove these features from the df 
df = df.drop(['pH','Temperature','Substrate Concentration'],axis=1)
df.head()

,ChEMBL ID,SMILES,EC50,EC50 units,Assay Type,Target Name
0,CHEMBL87897,COc1ccc(CC2Cc3c(ccc(O)c3O)[C@H](CN)O2)cc1,237.0,nM,F,Dopamine D1 receptor
1,CHEMBL2115213,CC[C@@H]1Cc2c(ccc(O)c2O)[C@H](CN)O1,142.0,nM,F,Dopamine D1 receptor
2,CHEMBL420972,CCCCCCC1Cc2c(ccc(O)c2O)[C@H](CN)O1,13.6,nM,F,Dopamine D1 receptor
3,CHEMBL420788,C#CCNC[C@@H]1OC(C2CCCCC2)Cc2c1ccc(O)c2O,12.5,nM,F,Dopamine D1 receptor
4,CHEMBL87910,NC[C@@H]1OC(Cc2ccccc2)Cc2c1ccc(O)c2O,34.4,nM,F,Dopamine D1 receptor


In [ ]:
df['Target Name'].unique()

array(['Dopamine D1 receptor', 'Dopamine D2 receptor',
       'Dopamine D3 receptor', 'Dopamine D4 receptor',
       'Dopamine D5 receptor'], dtype=object)

In [ ]:
# dataframe with rows of missing ec50 removed 

df_ec50 = df[df['EC50'].notnull()] 
df_ec50.head()

,ChEMBL ID,SMILES,EC50,EC50 units,Assay Type,Target Name
0,CHEMBL87897,COc1ccc(CC2Cc3c(ccc(O)c3O)[C@H](CN)O2)cc1,237.0,nM,F,Dopamine D1 receptor
1,CHEMBL2115213,CC[C@@H]1Cc2c(ccc(O)c2O)[C@H](CN)O1,142.0,nM,F,Dopamine D1 receptor
2,CHEMBL420972,CCCCCCC1Cc2c(ccc(O)c2O)[C@H](CN)O1,13.6,nM,F,Dopamine D1 receptor
3,CHEMBL420788,C#CCNC[C@@H]1OC(C2CCCCC2)Cc2c1ccc(O)c2O,12.5,nM,F,Dopamine D1 receptor
4,CHEMBL87910,NC[C@@H]1OC(Cc2ccccc2)Cc2c1ccc(O)c2O,34.4,nM,F,Dopamine D1 receptor


In [ ]:
# check for missing data now 
df_ec50.isna().sum()

ChEMBL ID       0
SMILES         15
EC50            0
EC50 units     19
Assay Type      0
Target Name     0
dtype: int64

In [ ]:
df_ec50['EC50 units'].value_counts()

EC50 units
nM    3243
Name: count, dtype: int64

In [ ]:
# we can assume that the missing units of EC50 are nM and we can remove this columnn as well. 
df_ec50 = df_ec50.drop('EC50 units', axis=1)
df_ec50.head()

,ChEMBL ID,SMILES,EC50,Assay Type,Target Name
0,CHEMBL87897,COc1ccc(CC2Cc3c(ccc(O)c3O)[C@H](CN)O2)cc1,237.0,F,Dopamine D1 receptor
1,CHEMBL2115213,CC[C@@H]1Cc2c(ccc(O)c2O)[C@H](CN)O1,142.0,F,Dopamine D1 receptor
2,CHEMBL420972,CCCCCCC1Cc2c(ccc(O)c2O)[C@H](CN)O1,13.6,F,Dopamine D1 receptor
3,CHEMBL420788,C#CCNC[C@@H]1OC(C2CCCCC2)Cc2c1ccc(O)c2O,12.5,F,Dopamine D1 receptor
4,CHEMBL87910,NC[C@@H]1OC(Cc2ccccc2)Cc2c1ccc(O)c2O,34.4,F,Dopamine D1 receptor


In [ ]:
# let us rename the EC50 feature with its units for better presentation 
df_ec50 = df_ec50.rename(columns={'EC50': 'EC50 (nM)'})
df_ec50.head()

,ChEMBL ID,SMILES,EC50 (nM),Assay Type,Target Name
0,CHEMBL87897,COc1ccc(CC2Cc3c(ccc(O)c3O)[C@H](CN)O2)cc1,237.0,F,Dopamine D1 receptor
1,CHEMBL2115213,CC[C@@H]1Cc2c(ccc(O)c2O)[C@H](CN)O1,142.0,F,Dopamine D1 receptor
2,CHEMBL420972,CCCCCCC1Cc2c(ccc(O)c2O)[C@H](CN)O1,13.6,F,Dopamine D1 receptor
3,CHEMBL420788,C#CCNC[C@@H]1OC(C2CCCCC2)Cc2c1ccc(O)c2O,12.5,F,Dopamine D1 receptor
4,CHEMBL87910,NC[C@@H]1OC(Cc2ccccc2)Cc2c1ccc(O)c2O,34.4,F,Dopamine D1 receptor


In [ ]:
df_null_smiles = df_ec50[df_ec50['SMILES'].isnull()] 

In [ ]:
df_null_smiles['ChEMBL ID'].value_counts()

ChEMBL ID
CHEMBL364516    3
CHEMBL194555    3
CHEMBL191962    3
CHEMBL370297    3
CHEMBL195083    3
Name: count, dtype: int64

In [ ]:
df_null_smiles

,ChEMBL ID,SMILES,EC50 (nM),Assay Type,Target Name
2467,CHEMBL364516,None,2.0,F,Dopamine D3 receptor
2468,CHEMBL194555,None,3.5,F,Dopamine D3 receptor
2469,CHEMBL191962,None,3.9,F,Dopamine D3 receptor
2470,CHEMBL370297,None,9.1,F,Dopamine D3 receptor
2471,CHEMBL195083,None,9.8,F,Dopamine D3 receptor
3293,CHEMBL364516,None,0.55,F,Dopamine D4 receptor
3294,CHEMBL364516,None,2.5,F,Dopamine D4 receptor
3295,CHEMBL194555,None,7.6,F,Dopamine D4 receptor
3296,CHEMBL194555,None,13.0,F,Dopamine D4 receptor
3297,CHEMBL191962,None,1.2,F,Dopamine D4 receptor


In [ ]:
# since there are only 5 total ChEMBL IDs with their SMILES missing, 
# I will manually fill these missing SMILES in the dataframe 
# The SMILES were obtained from BindingDB: https://137.110.139.247/rwd/bind/index.jsp

# CHEMBL364516 -> [Fe]123456789C%10C1=C2C3=C4%10.N(CCCCN1CCN(CC1)c1c(cccc1)OC)C(C51C6=C7C8=C91)=O
# CHEMBL194555 -> [Fe]123456789C%10=C1C2C3=C4%10.Clc1c(Cl)c(ccc1)N1CCN(CC1)CCCCNC(C51C6=C7C8=C91)=O
# CHEMBL191962 -> [Ru]123456789C%10C1=C2C3=C4%10.N(CCCCN1CCN(CC1)c1c(cccc1)OC)C(C51C6=C7C8=C91)=O
# CHEMBL370297 -> [Ru]123456789C%10=C1C2C3=C4%10.Clc1c(Cl)c(ccc1)N1CCN(CC1)CCCCNC(C51C6=C7C8=C91)=O
# CHEMBL195083 -> c1(Cl)c(c(N2CCN(CC2)CCCCNC(C23[Fe]456789%10%11C%12C4=C5C6=C7%12)=O)ccc1)OC.C8(C9=C2%10)=C3%11

def fill_smiles(chembl_id): 
    if chembl_id == 'CHEMBL364516': 
        return '[Fe]123456789C%10C1=C2C3=C4%10.N(CCCCN1CCN(CC1)c1c(cccc1)OC)C(C51C6=C7C8=C91)=O'
    elif chembl_id == 'CHEMBL194555': 
        return '[Fe]123456789C%10=C1C2C3=C4%10.Clc1c(Cl)c(ccc1)N1CCN(CC1)CCCCNC(C51C6=C7C8=C91)=O' 
    elif chembl_id == 'CHEMBL191962': 
        return '[Ru]123456789C%10C1=C2C3=C4%10.N(CCCCN1CCN(CC1)c1c(cccc1)OC)C(C51C6=C7C8=C91)=O'
    elif chembl_id == 'CHEMBL370297': 
        return '[Ru]123456789C%10=C1C2C3=C4%10.Clc1c(Cl)c(ccc1)N1CCN(CC1)CCCCNC(C51C6=C7C8=C91)=O'
    elif chembl_id == 'CHEMBL195083': 
        return 'c1(Cl)c(c(N2CCN(CC2)CCCCNC(C23[Fe]456789%10%11C%12C4=C5C6=C7%12)=O)ccc1)OC.C8(C9=C2%10)=C3%11'


In [ ]:
df_ec50['SMILES'] = df_ec50['SMILES'].fillna(df_ec50['ChEMBL ID'].apply(fill_smiles))

In [ ]:
df_ec50.isna().sum()

ChEMBL ID      0
SMILES         0
EC50 (nM)      0
Assay Type     0
Target Name    0
dtype: int64

#### Our dataset is now ready with no missing values. We can now: 
1. introduce a pEC50 column
2. enrich our dataframe using SMILES strings 

In [ ]:
# necessary imports 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd 
import requests 

%matplotlib inline 

In [ ]:
df_ec50.to_csv('extracted_dopamine.csv',index=False)

In [ ]:
df_ec50 = pd.read_csv('extracted_dopamine.csv')
df_ec50.head()

,ChEMBL ID,SMILES,EC50 (nM),Assay Type,Target Name
0,CHEMBL87897,COc1ccc(CC2Cc3c(ccc(O)c3O)[C@H](CN)O2)cc1,237.0,F,Dopamine D1 receptor
1,CHEMBL2115213,CC[C@@H]1Cc2c(ccc(O)c2O)[C@H](CN)O1,142.0,F,Dopamine D1 receptor
2,CHEMBL420972,CCCCCCC1Cc2c(ccc(O)c2O)[C@H](CN)O1,13.6,F,Dopamine D1 receptor
3,CHEMBL420788,C#CCNC[C@@H]1OC(C2CCCCC2)Cc2c1ccc(O)c2O,12.5,F,Dopamine D1 receptor
4,CHEMBL87910,NC[C@@H]1OC(Cc2ccccc2)Cc2c1ccc(O)c2O,34.4,F,Dopamine D1 receptor


In [ ]:
df_ec50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3262 entries, 0 to 3261
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ChEMBL ID    3262 non-null   object 
 1   SMILES       3262 non-null   object 
 2   EC50 (nM)    3262 non-null   float64
 3   Assay Type   3262 non-null   object 
 4   Target Name  3262 non-null   object 
dtypes: float64(1), object(4)
memory usage: 127.6+ KB


In [ ]:
df_ec50['pEC50'] = df_ec50['EC50 (nM)'].apply(lambda x: -1*(np.log10(x * 10**(-9))) )
df_ec50.head()


/var/folders/qn/18pqdj6j4y5bmd8j80dlz0n80000gn/T/ipykernel_3129/1714392294.py:1: RuntimeWarning: divide by zero encountered in log10
  df_ec50['pEC50'] = df_ec50['EC50 (nM)'].apply(lambda x: -1*(np.log10(x * 10**(-9))) )


,ChEMBL ID,SMILES,EC50 (nM),Assay Type,Target Name,pEC50
0,CHEMBL87897,COc1ccc(CC2Cc3c(ccc(O)c3O)[C@H](CN)O2)cc1,237.0,F,Dopamine D1 receptor,6.625252
1,CHEMBL2115213,CC[C@@H]1Cc2c(ccc(O)c2O)[C@H](CN)O1,142.0,F,Dopamine D1 receptor,6.847712
2,CHEMBL420972,CCCCCCC1Cc2c(ccc(O)c2O)[C@H](CN)O1,13.6,F,Dopamine D1 receptor,7.866461
3,CHEMBL420788,C#CCNC[C@@H]1OC(C2CCCCC2)Cc2c1ccc(O)c2O,12.5,F,Dopamine D1 receptor,7.903090
4,CHEMBL87910,NC[C@@H]1OC(Cc2ccccc2)Cc2c1ccc(O)c2O,34.4,F,Dopamine D1 receptor,7.463442


The warning says that an EC50 value is zero! Odd. Let's check and confirm where is it.

In [ ]:
df_ec50[df_ec50['EC50 (nM)']<0.001]

,ChEMBL ID,SMILES,EC50 (nM),Assay Type,Target Name,pEC50
2624,CHEMBL4294744,O=C1c2ccccc2C(=O)N1CCCCN1CCN(c2cccc(F)c2F)CC1,0.00048,F,Dopamine D3 receptor,12.318759
3197,CHEMBL4579981,Cc1ccc2[nH]c(CNCCCc3ccncc3)cc(=O)c2c1,0.00000,F,Dopamine D4 receptor,inf


We must remove index 3197 from our dataset which has EC50 = 0 nM. Potentially some error. 

In [ ]:
df_ec50 = df_ec50.drop(3197,axis=0).reset_index().drop('index',axis=1) 
df_ec50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3261 entries, 0 to 3260
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ChEMBL ID    3261 non-null   object 
 1   SMILES       3261 non-null   object 
 2   EC50 (nM)    3261 non-null   float64
 3   Assay Type   3261 non-null   object 
 4   Target Name  3261 non-null   object 
 5   pEC50        3261 non-null   float64
dtypes: float64(2), object(4)
memory usage: 153.0+ KB


### Time to enrich our dataset using the SMILES strings now. 

In [ ]:
# before that, are there any duplicates? 
duplicate = df_ec50[df_ec50.duplicated()]
duplicate

,ChEMBL ID,SMILES,EC50 (nM),Assay Type,Target Name,pEC50
57,CHEMBL86931,NC[C@@H]1O[C@H](c2ccccc2)Cc2c1ccc(O)c2O,1.95,F,Dopamine D1 receptor,8.709965
58,CHEMBL83080,NC[C@H]1O[C@@H](c2ccccc2)Cc2c1ccc(O)c2O,8577.00,F,Dopamine D1 receptor,5.066665
62,CHEMBL286080,Oc1cc2c(cc1O)C(c1ccccc1)CNCC2,386.00,F,Dopamine D1 receptor,6.413413
63,CHEMBL86931,NC[C@@H]1O[C@H](c2ccccc2)Cc2c1ccc(O)c2O,2.10,F,Dopamine D1 receptor,8.677781
64,CHEMBL86931,NC[C@@H]1O[C@H](c2ccccc2)Cc2c1ccc(O)c2O,1.95,F,Dopamine D1 receptor,8.709965
...,...,...,...,...,...,...
2976,CHEMBL240773,CCCN1CCC[C@@H]2Cc3[nH]ncc3C[C@H]21,49.00,F,Dopamine D4 receptor,7.309804
3104,CHEMBL45244,Cc1cccc(C(=O)NCN2CCN(c3ccccc3C#N)CC2)c1,8.30,F,Dopamine D4 receptor,8.080922
3167,CHEMBL240773,CCCN1CCC[C@@H]2Cc3[nH]ncc3C[C@H]21,2.20,F,Dopamine D4 receptor,8.657577
3194,CHEMBL440687,c1ccc(N2CCN(Cc3nc4ccccc4[nH]3)CC2)nc1,12.40,B,Dopamine D4 receptor,7.906578


We should remove these duplicated rows from our dataset. 

In [ ]:
clean_df_ec50 = df_ec50.drop_duplicates().reset_index().drop('index',axis=1)  
clean_df_ec50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3125 entries, 0 to 3124
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ChEMBL ID    3125 non-null   object 
 1   SMILES       3125 non-null   object 
 2   EC50 (nM)    3125 non-null   float64
 3   Assay Type   3125 non-null   object 
 4   Target Name  3125 non-null   object 
 5   pEC50        3125 non-null   float64
dtypes: float64(2), object(4)
memory usage: 146.6+ KB


Now, onto enriching our dataset using SMILES strings.

In [ ]:
# extracting molecular descriptors using RDKit

from rdkit import Chem
from rdkit.Chem import Descriptors

# Defining a function to calculate molecular descriptors
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mw = Descriptors.MolWt(mol)  # Molecular Weight
    logp = Descriptors.MolLogP(mol)  # LogP
    h_donors = Descriptors.NumHDonors(mol)  # Hydrogen bond donors
    h_acceptors = Descriptors.NumHAcceptors(mol)  # Hydrogen bond acceptors
    tpsa = Descriptors.TPSA(mol)  # Topological Polar Surface Area
    ring_count = Descriptors.RingCount(mol)  # Ring count
    rotatable_bonds = Descriptors.NumRotatableBonds(mol)  # Rotatable bonds
    return pd.Series([mw, logp, h_donors, h_acceptors, tpsa, ring_count, rotatable_bonds],
                             index=['MW', 'LogP', 'H_Donors', 'H_Acceptors', 'TPSA', 'Ring_Count', 'Rotatable_Bonds'])

# Applying the descriptor calculation function to the SMILES column and storing in a dataframe
mol_desc_df = clean_df_ec50['SMILES'].apply(calculate_descriptors)


# To display the updated DataFrame
mol_desc_df.head()

,MW,LogP,H_Donors,H_Acceptors,TPSA,Ring_Count,Rotatable_Bonds
0,315.369,2.2902,3.0,5.0,84.94,3.0,4.0
1,223.272,1.4489,3.0,4.0,75.71,2.0,2.0
2,279.380,3.0093,3.0,4.0,75.71,2.0,6.0
3,315.413,2.8833,3.0,4.0,61.72,3.0,4.0
4,285.343,2.2816,3.0,4.0,75.71,3.0,3.0


In [ ]:
# combine both the dataframes now 

enriched_df_ec50 = pd.concat([clean_df_ec50,mol_desc_df],axis=1)
enriched_df_ec50.head()

,ChEMBL ID,SMILES,EC50 (nM),Assay Type,Target Name,pEC50,MW,LogP,H_Donors,H_Acceptors,TPSA,Ring_Count,Rotatable_Bonds
0,CHEMBL87897,COc1ccc(CC2Cc3c(ccc(O)c3O)[C@H](CN)O2)cc1,237.0,F,Dopamine D1 receptor,6.625252,315.369,2.2902,3.0,5.0,84.94,3.0,4.0
1,CHEMBL2115213,CC[C@@H]1Cc2c(ccc(O)c2O)[C@H](CN)O1,142.0,F,Dopamine D1 receptor,6.847712,223.272,1.4489,3.0,4.0,75.71,2.0,2.0
2,CHEMBL420972,CCCCCCC1Cc2c(ccc(O)c2O)[C@H](CN)O1,13.6,F,Dopamine D1 receptor,7.866461,279.380,3.0093,3.0,4.0,75.71,2.0,6.0
3,CHEMBL420788,C#CCNC[C@@H]1OC(C2CCCCC2)Cc2c1ccc(O)c2O,12.5,F,Dopamine D1 receptor,7.903090,315.413,2.8833,3.0,4.0,61.72,3.0,4.0
4,CHEMBL87910,NC[C@@H]1OC(Cc2ccccc2)Cc2c1ccc(O)c2O,34.4,F,Dopamine D1 receptor,7.463442,285.343,2.2816,3.0,4.0,75.71,3.0,3.0


In [ ]:
enriched_df_ec50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3125 entries, 0 to 3124
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ChEMBL ID        3125 non-null   object 
 1   SMILES           3125 non-null   object 
 2   EC50 (nM)        3125 non-null   float64
 3   Assay Type       3125 non-null   object 
 4   Target Name      3125 non-null   object 
 5   pEC50            3125 non-null   float64
 6   MW               3125 non-null   float64
 7   LogP             3125 non-null   float64
 8   H_Donors         3125 non-null   float64
 9   H_Acceptors      3125 non-null   float64
 10  TPSA             3125 non-null   float64
 11  Ring_Count       3125 non-null   float64
 12  Rotatable_Bonds  3125 non-null   float64
dtypes: float64(9), object(4)
memory usage: 317.5+ KB


#### Our first section is now complete with the creation of the dataset. We need to clean the dataset now for our further analysis.

## 2. Cleaning and preprocessing our data. Making sure our target label is pEC50.

In [ ]:
# now, we can remove the ChEMBL ID and SMILES feature from our dataset. 
# Also, we should remove the EC50 (nM) column as well because we have the pEC50 values now. 

final_df_ec50 = enriched_df_ec50.drop(['ChEMBL ID','SMILES','EC50 (nM)'],axis=1)
final_df_ec50.head()

,Assay Type,Target Name,pEC50,MW,LogP,H_Donors,H_Acceptors,TPSA,Ring_Count,Rotatable_Bonds
0,F,Dopamine D1 receptor,6.625252,315.369,2.2902,3.0,5.0,84.94,3.0,4.0
1,F,Dopamine D1 receptor,6.847712,223.272,1.4489,3.0,4.0,75.71,2.0,2.0
2,F,Dopamine D1 receptor,7.866461,279.380,3.0093,3.0,4.0,75.71,2.0,6.0
3,F,Dopamine D1 receptor,7.903090,315.413,2.8833,3.0,4.0,61.72,3.0,4.0
4,F,Dopamine D1 receptor,7.463442,285.343,2.2816,3.0,4.0,75.71,3.0,3.0


In [ ]:
# saving the original enriched and final df to csvs now 
enriched_df_ec50.to_csv("enriched_dopamine_ec50.csv",index=False)
final_df_ec50.to_csv("dopamine_pEC50.csv",index=False) 

### The final dataset with the enriched parameters is ready and we can start with our EDA and machine learning now.
### There are no missing values, no duplicates and the data is all set. 


## 2 (a). About the Data

* For all 5 different types of dopamine receptors: D1 to D5, a dataset was constructed consisting of small molecules that interact with these receptors and their respective EC50 values. The following information was extracted from ChEMBL:
    * ChEMBL ID of the molecule
    * SMILES string
    * target information
    * EC50 values (nM units)
* For a more convinent study in practical terms, **pEC50** was determined from EC50 values. 
* Later, the dataset was enriched with the molecular descriptors of these small molecules using the SMILES information:
   * MW
   * LogP
   * Number of H donors and acceptors
   * Topological Polar Surface Area (TPSA)
   * Ring count: Number of rings in the molecule.
   * Rotatable bonds: Number of bonds that can freely rotate.
* Finally, the ChEMBL IDs, SMILES strings and EC50 features were removed from the dataset afterwards. 


The data dictionary, now, is as follows: 

| S. No. | Feature | Numerical/Categorical | Description | Units (if any) or any additional information | 
| --- | --- | --- | --- | --- | 
| 1. | Assay Type | C | B (binding), F (functional), A (ADME) | B: how a compound binds to a molecular target; F: measures the biological effect/activity of a compound; A: Absorption, Distribution, Metabolism, and Excretion of a drug. A critical part of drug discovery and development because they help ensure that a drug is safe and effective. | 
| 2. | Target Name | C | The type of dopamine receptor that the molecule targets. | Dopamine D1/D2/D3/D4/D5 receptor |
| 3. | pEC50 | N | - log10(EC50) value | Another way EC50 values can be presented | 
| 4. | MW | N | Molecular weight | g/mol |
| 5. | LogP | N | Water-octanol partition coefficient |  |
| 6. | H_Donors | N | Number of Hydrogen bond donors |  |
| 7. | H_Acceptors | N | Number of Hydrogen bonds acceptors |  |
| 8. | TPSA | N | Topological Polar Surface Area: A metric used to predict how well a drug can permeate cells and reach its target sites in the body. TPSA value increases as the number of polar groups (containing N and O) within the drug structure increases. TPSA <= 140 Å2 or less is considered good for oral bioavailability (for its target). | Ångströms squared (Å^2) |
| 9. | Ring count | N | Number of rings in the molecule |  |
| 10. | Rotatable bonds | N | Number of bonds that can freely rotate |  |


In [ ]:
df = pd.read_csv('dopamine_pEC50.csv')
df.head()

,Assay Type,Target Name,pEC50,MW,LogP,H_Donors,H_Acceptors,TPSA,Ring_Count,Rotatable_Bonds
0,F,Dopamine D1 receptor,6.625252,315.369,2.2902,3.0,5.0,84.94,3.0,4.0
1,F,Dopamine D1 receptor,6.847712,223.272,1.4489,3.0,4.0,75.71,2.0,2.0
2,F,Dopamine D1 receptor,7.866461,279.380,3.0093,3.0,4.0,75.71,2.0,6.0
3,F,Dopamine D1 receptor,7.903090,315.413,2.8833,3.0,4.0,61.72,3.0,4.0
4,F,Dopamine D1 receptor,7.463442,285.343,2.2816,3.0,4.0,75.71,3.0,3.0


In [ ]:
df['Assay Type'].value_counts()

Assay Type
F    2168
B     822
A     135
Name: count, dtype: int64

In [ ]:
# better to rename Target Name column values to shorter names. 

df['Target Name'] = df['Target Name'].replace({'Dopamine D1 receptor':'D1','Dopamine D2 receptor':'D2',
                                              'Dopamine D3 receptor':'D3','Dopamine D4 receptor':'D4', 'Dopamine D5 receptor':'D5'})
df.head()

,Assay Type,Target Name,pEC50,MW,LogP,H_Donors,H_Acceptors,TPSA,Ring_Count,Rotatable_Bonds
0,F,D1,6.625252,315.369,2.2902,3.0,5.0,84.94,3.0,4.0
1,F,D1,6.847712,223.272,1.4489,3.0,4.0,75.71,2.0,2.0
2,F,D1,7.866461,279.380,3.0093,3.0,4.0,75.71,2.0,6.0
3,F,D1,7.903090,315.413,2.8833,3.0,4.0,61.72,3.0,4.0
4,F,D1,7.463442,285.343,2.2816,3.0,4.0,75.71,3.0,3.0


### The above reflects the final dataset that we will be working with. 